### 1. Add Liberaries


In [248]:
import pandas as pd
import glob
import requests
import urllib, json

### 2. Read and explore the dataset

In [249]:
#Read and explore the dataset
ttc_df = pd.read_excel("ttc_data.xlsx")

In [250]:
ttc_df

,Unnamed: 0,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle
0,0,2014-01-01,00:21,Wednesday,VICTORIA PARK STATION,MUPR1,55,60,W,BD,5111
1,1,2014-01-01,02:06,Wednesday,HIGH PARK STATION,SUDP,3,7,W,BD,5001
2,2,2014-01-01,02:40,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0
3,3,2014-01-01,03:10,Wednesday,LANSDOWNE STATION,SUDP,3,8,W,BD,5116
4,4,2014-01-01,03:20,Wednesday,BLOOR STATION,MUSAN,5,10,S,YU,5386
5,5,2014-01-01,03:29,Wednesday,DUFFERIN STATION,MUPAA,0,0,E,BD,5174
6,6,2014-01-01,07:31,Wednesday,NORTH YORK CTR STATION,MUNCA,0,0,NaN,YU,0
7,7,2014-01-01,07:32,Wednesday,SHEPPARD STATION,MUNCA,0,0,NaN,YU,0
8,8,2014-01-01,07:34,Wednesday,QUEEN STATION,MUNCA,0,0,NaN,YU,0
9,9,2014-01-01,07:34,Wednesday,RUNNYMEDE STATION,MUNCA,0,0,NaN,BD,0


### 3. Feature Engineering 

After exploration we can add more features to this dataset. Following is the list of features we can add:

- Separate the Bounds (East, West,North,South)
- Separate Weekend Delays (Weekend Sat, Weend Sun)
- Isolating the month, year, and day of the week as separate features (Day,Month,Year)
- Add City Column to the dataset
- Get part of the day (Morning, Afternoon, Evening,Night)
- Addt Hour and Minute column from Timestamp
- Add latitude and longitude (lat,lon)

In [257]:
#Separate bounds
ttc_df["East Bound"] = ttc_df["Bound"] == 'E'
ttc_df["West Bound"] =ttc_df["Bound"] == 'W'
ttc_df["North Bound"] = ttc_df["Bound"] == 'N'
ttc_df["South Bound"] = ttc_df["Bound"] == 'S'

# Separate Weekend Delays
ttc_df["Weekend-Sat"] = ttc_df["Day"] == 'Saturday'
ttc_df["Weekend-Sun"] = ttc_df["Day"] == 'Sunday'

#Isolating the month, year, and day of the week as separate predictors
#The numeric day of the year (ignoring the calendar year)
#if there were a seasonal component to these data, and it appears
#that there is, then the numeric day of the year would be best. Also, if some
#months showed higher success rates than others those should be taken into further EDA
#the month is preferable.

ttc_df['Year'] = pd.DatetimeIndex(ttc_df['Date']).year
ttc_df['Month'] = pd.DatetimeIndex(ttc_df['Date']).month
ttc_df['Day'] = pd.DatetimeIndex(ttc_df['Date']).day

#add city column
ttc_df['City'] = 'Toronto'
ttc_df

,Unnamed: 0,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,...,North Bound,South Bound,Weekend-Sat,Weekend-Sun,Year,Month,City,Hour,Minute,Time-only
0,0,2014-01-01,00:21,1,VICTORIA PARK STATION,MUPR1,55,60,W,BD,...,False,False,False,False,2014,1,Toronto,0,21,Late Night
1,1,2014-01-01,02:06,1,HIGH PARK STATION,SUDP,3,7,W,BD,...,False,False,False,False,2014,1,Toronto,2,6,Late Night
2,2,2014-01-01,02:40,1,SHEPPARD STATION,MUNCA,0,0,NaN,YU,...,False,False,False,False,2014,1,Toronto,2,40,Late Night
3,3,2014-01-01,03:10,1,LANSDOWNE STATION,SUDP,3,8,W,BD,...,False,False,False,False,2014,1,Toronto,3,10,Late Night
4,4,2014-01-01,03:20,1,BLOOR STATION,MUSAN,5,10,S,YU,...,False,True,False,False,2014,1,Toronto,3,20,Late Night
5,5,2014-01-01,03:29,1,DUFFERIN STATION,MUPAA,0,0,E,BD,...,False,False,False,False,2014,1,Toronto,3,29,Late Night
6,6,2014-01-01,07:31,1,NORTH YORK CTR STATION,MUNCA,0,0,NaN,YU,...,False,False,False,False,2014,1,Toronto,7,31,Early Morning
7,7,2014-01-01,07:32,1,SHEPPARD STATION,MUNCA,0,0,NaN,YU,...,False,False,False,False,2014,1,Toronto,7,32,Early Morning
8,8,2014-01-01,07:34,1,QUEEN STATION,MUNCA,0,0,NaN,YU,...,False,False,False,False,2014,1,Toronto,7,34,Early Morning
9,9,2014-01-01,07:34,1,RUNNYMEDE STATION,MUNCA,0,0,NaN,BD,...,False,False,False,False,2014,1,Toronto,7,34,Early Morning


In [258]:
ttc_df.dtypes

Unnamed: 0              int64
Date           datetime64[ns]
Time                   object
Day                     int64
Station                object
Code                   object
Min Delay               int64
Min Gap                 int64
Bound                  object
Line                   object
Vehicle                 int64
East Bound               bool
West Bound               bool
North Bound              bool
South Bound              bool
Weekend-Sat              bool
Weekend-Sun              bool
Year                    int64
Month                   int64
City                   object
Hour                    int64
Minute                  int64
Time-only              object
dtype: object

In [252]:
ttc_df[['Hour','Minute']] = ttc_df['Time'].astype(str).str.split(':', expand=True).astype(int)
ttc_df

,Unnamed: 0,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,...,West Bound,North Bound,South Bound,Weekend-Sat,Weekend-Sun,Year,Month,City,Hour,Minute
0,0,2014-01-01,00:21,1,VICTORIA PARK STATION,MUPR1,55,60,W,BD,...,True,False,False,False,False,2014,1,Toronto,0,21
1,1,2014-01-01,02:06,1,HIGH PARK STATION,SUDP,3,7,W,BD,...,True,False,False,False,False,2014,1,Toronto,2,6
2,2,2014-01-01,02:40,1,SHEPPARD STATION,MUNCA,0,0,NaN,YU,...,False,False,False,False,False,2014,1,Toronto,2,40
3,3,2014-01-01,03:10,1,LANSDOWNE STATION,SUDP,3,8,W,BD,...,True,False,False,False,False,2014,1,Toronto,3,10
4,4,2014-01-01,03:20,1,BLOOR STATION,MUSAN,5,10,S,YU,...,False,False,True,False,False,2014,1,Toronto,3,20
5,5,2014-01-01,03:29,1,DUFFERIN STATION,MUPAA,0,0,E,BD,...,False,False,False,False,False,2014,1,Toronto,3,29
6,6,2014-01-01,07:31,1,NORTH YORK CTR STATION,MUNCA,0,0,NaN,YU,...,False,False,False,False,False,2014,1,Toronto,7,31
7,7,2014-01-01,07:32,1,SHEPPARD STATION,MUNCA,0,0,NaN,YU,...,False,False,False,False,False,2014,1,Toronto,7,32
8,8,2014-01-01,07:34,1,QUEEN STATION,MUNCA,0,0,NaN,YU,...,False,False,False,False,False,2014,1,Toronto,7,34
9,9,2014-01-01,07:34,1,RUNNYMEDE STATION,MUNCA,0,0,NaN,BD,...,False,False,False,False,False,2014,1,Toronto,7,34


In [254]:
#Craete a function to get part of the day(morning, noon,afternoon evening and night) from Timestamp
def f(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Eve'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'
    

In [256]:
ttc_df['Time-only'] = ttc_df['Hour'].apply(f)
ttc_df['Time-only']

0            Late Night
1            Late Night
2            Late Night
3            Late Night
4            Late Night
5            Late Night
6         Early Morning
7         Early Morning
8         Early Morning
9         Early Morning
10        Early Morning
11        Early Morning
12        Early Morning
13        Early Morning
14        Early Morning
15        Early Morning
16        Early Morning
17              Morning
18              Morning
19              Morning
20              Morning
21              Morning
22              Morning
23              Morning
24              Morning
25              Morning
26              Morning
27              Morning
28              Morning
29              Morning
              ...      
112483          Morning
112484          Morning
112485          Morning
112486          Morning
112487          Morning
112488             Noon
112489             Noon
112490             Noon
112491             Noon
112492             Noon
112493          

In [229]:
len(ttc_df['Station'].unique())

464

In [260]:
#Get the list of all stations in dataset
stations = list(ttc_df['Station'].unique())
stations


['VICTORIA PARK STATION',
 'HIGH PARK STATION',
 'SHEPPARD STATION',
 'LANSDOWNE STATION',
 'BLOOR STATION',
 'DUFFERIN STATION',
 'NORTH YORK CTR STATION',
 'QUEEN STATION',
 'RUNNYMEDE STATION',
 'ST ANDREW STATION',
 'WELLESLEY STATION',
 'KIPLING STATION',
 'YONGE SHP STATION',
 'ST PATRICK STATION',
 'KENNEDY BD STATION',
 'JANE STATION',
 'OLD MILL STATION',
 'WILSON CARHOUSE',
 'SHEPPARD WEST STATION',
 'BAY STATION',
 'FINCH STATION',
 'COXWELL STATION',
 'ST CLAIR WEST STATION',
 'DUPONT STATION',
 'YORK MILLS STATION',
 'DUNDAS WEST STATION',
 'OSGOODE STATION',
 'COLLEGE STATION',
 'YONGE BD STATION',
 'ST GEORGE YUS STATION',
 'MCCOWAN STATION',
 'BROADVIEW STATION',
 'EGLINTON STATION',
 'ROSEDALE STATION',
 'PAPE STATION',
 'GREENWOOD STATION',
 'LAWRENCE EAST STATION',
 'KENNEDY SRT STATION',
 'ISLINGTON STATION',
 'WARDEN STATION',
 'DAVISVILLE STATION',
 'LAWRENCE STATION',
 'EGLINTON WEST STATION',
 'KEELE STATION',
 'MIDLAND STATION',
 'YONGE UNIVERSITY LINE',
 'ELLE

In [259]:
import requests
import urllib, json
#address = ["1600+Amphitheatre+Parkway,+Mountain+View,+CA"]
#url = "http://www.datasciencetoolkit.org/maps/api/geocode/json?sensor=false&address=1600+Amphitheatre+Parkway,+Mountain+View,+CA"

# replace space with +sign to create url endpoint

url = "http://www.datasciencetoolkit.org/maps/api/geocode/json?sensor=false&address=1600+Amphitheatre+Parkway,+Mountain+View,+CA"

data = requests.get(url).json()
type(data)
data.keys()
type(data['results'])
type(data['status'])
lat = data['results'][0]['geometry']['location']['lat']
lng = data['results'][0]['geometry']['location']['lng']
print(lat,lng)


37.423471 -122.086546


In [232]:
#data = ['MCCOWAN+STATION','SPADINA+YUS+STATION','MAIN+STREET+STATION']
#url = "http://www.datasciencetoolkit.org/maps/api/geocode/json?sensor=false&address=data,+TORONTO"

  
# api-endpoint 
#URL = "http://maps.googleapis.com/maps/api/geocode/json"
#URL = "http://www.datasciencetoolkit.org/maps/api/geocode/json?sensor=false&"
  
# location given here 
#location = "delhi technological university"
#stations = ttc_df['Station']
#stations = [x.replace(" ", "+") for x in stations]
  
# defining a params dict for the parameters to be sent to the API 
#PARAMS = {'sensor': False, 'address':stations} 
  
# sending get request and saving the response as response object 
#r = requests.get(url = URL, params = PARAMS)
  
# extracting data in json format 
#data = r.json() 
  
# extracting latitude, longitude and formatted address  of the first matching location 
#latitude = data['results'][0]['geometry']['location']['lat'] 
#longitude = data['results'][0]['geometry']['location']['lng'] 
#formatted_address = data['results'][0]['formatted_address'] 
  
#printing the output 
#print("Latitude:%s\nLongitude:%s\nFormatted Address:%s"
      #%(latitude, longitude,formatted_address)) 

In [236]:
# For creating URL endpoint replace space with + in the station names 
ttc_df['Stations'] = ttc_df['Station']
ttc_df['Stations'] = [x.replace(" ", "+") for x in stations]
#ttc_df['Station']

In [237]:
#create two empty lists for lat and lng
lat = []
lon = []

for index, row in ttc_df.iterrows():
    
    Station = row['Stations']
    City     = row['City']
    query    = str(Station)+','+str(City)
    URL = "http://www.datasciencetoolkit.org/maps/api/geocode/json?sensor=false&address="
    result = URL + query
    data = requests.get(result).json()

KeyboardInterrupt: 